#### This has code generating Hadamard matrices then produces a widget. This generates all
#### combination of patterns for different numbers of positive samples with various matrix sizes. 

### Use the 'cell' in the drop-down menu above then  'run all' to start the calculation. This will produce a   widget window at the end. Use this to produce vector/patterns. The code is written in Python 3 in a Jupyter Notebook running under Anaconda. Push GO on the widget.

This code is to only be used as a guide. No guarantee is given as to its accuracy. You should use this code only as an aid to writing your own algorithms.

#### Multiplex Detection of Viruses using Hadamard Matrices. G. S. Beddard & B. Yorke, School of Chemistry, University of Leeds, LS2 9JT, UK

https://www.medrxiv.org/content/10.1101/2024.10.21.24315883v2

Email. g.s.beddard@leeds.ac.uk, b.a.yorke@leeds.ac.uk


In [ ]:
# import all python 3 add-ons etc that will be needed later on
%matplotlib inline
import numpy as np
import numpy.linalg as La
import matplotlib.pyplot as plt
import time
from itertools import combinations
import ipywidgets.widgets  as wgt
from ipywidgets import interact, interactive, fixed, interact_manual,VBox,HBox,Layout,Output
from IPython.display import display

# Start of Hadamard matrix generation.
### Quadratic residue, Shift Register & Doubling and methods and special Legendre method for $H_{28}$

In [ ]:
#------------
def valid_seq_length(n):              # check Hadamard sequence length for Quadratic Residue method only
    maxi = 230
    Hseq = np.zeros(maxi,dtype=int)
    for i in range(maxi):                   # produce Hadamard sequence numbers
        for m in range(0,maxi):
            if isprime(i) and i == 4*m + 3:
                Hseq[i]=i
            pass
    if n in Hseq[0:maxi]:
        is_ok=True
    else:
        is_ok=False
    return is_ok
#------------
# check if integer n is a prime, range starts with 2 and only needs to go up the squareroot of n
def isprime(n):              
    for x in range(2, int(n**0.5)+1):
        if n % x == 0:
            return False
    return True
#------------
# quadratic residue method, this generates more S matrices than the other methods.
def quadratic_hadamard(n):    
    
    init_list = np.zeros(n,dtype=int)

    for i in range(n):                     # make list n/2+1 values = 1 rest zeros so in same ratio as hadamard
        if i <= n//2 : init_list[i] = 1    # check integer division
        pass
    alist = np.zeros(n,dtype=int)
    for i in range(0,(n-1)//2):            # integer division
        alist[(i+1)*(i+1) % n] = 1         # alist = hadamard Srow need only go to half range of n as indices are symmetric
    alist[0] = 1
    Srow = list(alist)
    
    S = np.zeros((n,n),dtype = int)
    for i in range(n):
        for j in range(n):
            S[i,j]= Srow[n-1-j]
        Srow = np.roll(Srow, 1)             # rotate by 1 element at a time
        pass
    
    return S                               # returns S matrix
#------------
#print('Hadamard S matrices by Quadratic residue method')
#for i in range(1,200):
#    if valid_seq_length(i):
#        S = quadratic_hadamard(i)
#        print(i)
#        if i  <= 32 :
#            print('\n'.join( [''.join(['{:2}'.format(item) for item in row] ) for row in S] ) )
#        else:
#            print(''.join(['{:2}'.format(item) for item in S[0]]))
#            xs=''.join( str(S[0][i]) for i in range(len(S[0])) )
#            print(hex(int(xs,2)))
#    pass 

In [ ]:
print( 'valid sequence with Quardatic residue method')
for i in range(130):
    if valid_seq_length(i) ==True:
        print(i,',',end='')

valid sequence with Quardatic residue method
0 ,3 ,7 ,11 ,19 ,23 ,31 ,43 ,47 ,59 ,67 ,71 ,79 ,83 ,103 ,107 ,127 ,

In [ ]:
#-------------
# shift register method S matrix size  2^k-1 , k = 2, 3, 4  etc
def shift_hadamard(num):                
    num = int(num) + 1
    def S_lineA(n,m):
        SL     = np.zeros(n,dtype=int)
        SL[0] = 1                               # set x^n = 1
        for j in range(2**n-1):

            tmp = (SL[m] + SL[0]) % 2           # mod 2
            SL = np.roll(SL,-1)                 # shift array elements
            SL[n-1] = tmp                       # last one as x^n
            Srow[j] = SL[0]
        pass

    def S_lineB(n,ma,mb,mc):
        SL    = np.zeros(n,dtype=int)
        SL[0] = 1                               # set x^n = 1
        for j in range(2**n-1):
            tmp1 = (SL[ma] + SL[0]) % 2         # mod 2
            tmp2 = (SL[mb] + tmp1 ) % 2
            tmp  = (SL[mc] + tmp2 ) % 2
            SL = np.roll(SL,-1)
            SL[n-1] = tmp                       # last one as x^n
            Srow[j] = SL[0]
        pass

    for k in range(num-1,num):                  # can generate v large matrices, 2^8-1, 2^9-1
        n = 2**k-1
        Srow = np.zeros(n,dtype=int)            # holds one row of S matrix
        if k in [2,3,4,6,7,15]:	S_lineA(k,1)
        if k in [5,11]:	S_lineA(k,2)
        if k == 8:      S_lineB(k,1,5,6)
        if k == 9:      S_lineA(k,4)
        if k == 10:     S_lineA(k,3)
        if k == 12:     S_lineB(k,3,4,7)
        if k == 13:     S_lineB(k,1,3,4)
        if k == 14:     S_lineB(k,1,11,12)
        pass

        S = np.zeros((n,n),dtype = int)
        for i in range(n):
            for j in range(n):
                S[i,j] = Srow[n-1-j]
            Srow = np.roll(Srow, 1)                   # rotate by 1 element at a time
            pass

    return S
#------------
# 2^k+1  is size of matrix side
S = shift_hadamard(4)
print(S)

[[1 1 1 1 0 1 0 1 1 0 0 1 0 0 0]
 [1 1 1 0 1 0 1 1 0 0 1 0 0 0 1]
 [1 1 0 1 0 1 1 0 0 1 0 0 0 1 1]
 [1 0 1 0 1 1 0 0 1 0 0 0 1 1 1]
 [0 1 0 1 1 0 0 1 0 0 0 1 1 1 1]
 [1 0 1 1 0 0 1 0 0 0 1 1 1 1 0]
 [0 1 1 0 0 1 0 0 0 1 1 1 1 0 1]
 [1 1 0 0 1 0 0 0 1 1 1 1 0 1 0]
 [1 0 0 1 0 0 0 1 1 1 1 0 1 0 1]
 [0 0 1 0 0 0 1 1 1 1 0 1 0 1 1]
 [0 1 0 0 0 1 1 1 1 0 1 0 1 1 0]
 [1 0 0 0 1 1 1 1 0 1 0 1 1 0 0]
 [0 0 0 1 1 1 1 0 1 0 1 1 0 0 1]
 [0 0 1 1 1 1 0 1 0 1 1 0 0 1 0]
 [0 1 1 1 1 0 1 0 1 1 0 0 1 0 0]]


In [ ]:
#--------------
# S matrix size (2^k) -1 , k = 2, 3, 4 etc, replace each element by previous matrix
def doubling_hadamard(max_size):    
    
    print('Matrix doubling method size (2^k)-1, k=2,3.. :  k=',max_size , 'S matrix is not circulant')
    max_size = int(max_size)
    h0 = np.ones( (2,2),dtype = int )     # define initial Hadamard matrix
    h0[1,0] = -1
    #h0[1,1]=-1    # this way round leading row & col are 1's 
    print('initial form\n',h0,h0 @ h0.T)
    n = 2  # must be 2
    Htemp = h0
    for i in range(1,max_size):
        print(' {:s} {:d}'.format(' calculation continues, S size = ',2*2**i-1))
        Hnn = np.zeros((2*n,2*n),dtype=int)
        h0, h1 = Htemp.shape
        Hnn[0  : 0  + h0   , 0  : 0  + h1] =  Htemp
        Hnn[n  : n  + h0   , 0  : 0  + h1] = -Htemp 
        Hnn[0  : 0  + h0   , n  : n  + h1] =  Htemp
        Hnn[n  : n  + h0   , n  : n  + h1] =  Htemp
        print('Hnn\n',Hnn)
        n = 2*n
        s = Hnn.shape
        Htemp = np.zeros( s ,dtype = int)
        Htemp = Hnn[ 0:s[0],  0:s[0] ]

        sm = s[0] - 1
        Tmpmat = np.zeros((sm,sm),dtype=int)
        Smat   = np.zeros((sm,sm),dtype=int)
        Tmpmat= Htemp[1:sm+1, 0:sm]
        for ii in range(sm):
            for j in range(sm):
                if Tmpmat[ii,j] == 1:
                    Smat[ii,j]= 0
                else:
                    Smat[ii,j]= 1
                pass
    print('-------------------------------')
    return Smat
#-------------

smat = doubling_hadamard(4)                   # 4 means  15 x 15 matrix
print('final S matrix \n',smat.shape)
print('\n'.join( [''.join(['{:2}'.format(item) for item in row] ) for row in smat] ) )

Matrix doubling method size (2^k)-1, k=2,3.. :  k= 4 S matrix is not circulant
initial form
 [[ 1  1]
 [-1  1]] [[2 0]
 [0 2]]
  calculation continues, S size =  3
Hnn
 [[ 1  1  1  1]
 [-1  1 -1  1]
 [-1 -1  1  1]
 [ 1 -1 -1  1]]
  calculation continues, S size =  7
Hnn
 [[ 1  1  1  1  1  1  1  1]
 [-1  1 -1  1 -1  1 -1  1]
 [-1 -1  1  1 -1 -1  1  1]
 [ 1 -1 -1  1  1 -1 -1  1]
 [-1 -1 -1 -1  1  1  1  1]
 [ 1 -1  1 -1 -1  1 -1  1]
 [ 1  1 -1 -1 -1 -1  1  1]
 [-1  1  1 -1  1 -1 -1  1]]
  calculation continues, S size =  15
Hnn
 [[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1]
 [-1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1]
 [-1 -1  1  1 -1 -1  1  1 -1 -1  1  1 -1 -1  1  1]
 [ 1 -1 -1  1  1 -1 -1  1  1 -1 -1  1  1 -1 -1  1]
 [-1 -1 -1 -1  1  1  1  1 -1 -1 -1 -1  1  1  1  1]
 [ 1 -1  1 -1 -1  1 -1  1  1 -1  1 -1 -1  1 -1  1]
 [ 1  1 -1 -1 -1 -1  1  1  1  1 -1 -1 -1 -1  1  1]
 [-1  1  1 -1  1 -1 -1  1 -1  1  1 -1  1 -1 -1  1]
 [-1 -1 -1 -1 -1 -1 -1 -1  1  1  1  1  1  1  1  1]
 [ 1 -1  

In [ ]:
#This Python code was originally based on the R code of 
#J. Steepleton, "Constructions of Hadamard Matrices" (2019). 
#Chancellor’s Honors Program Projects. https://trace.tennessee.edu/utk_chanhonoproj/2266

# matrix size;  m = 28  so divisible by 4 
# 2^k(p+1) = 28 when k=2, p=13

#-------------------------------
def legendre(p):                      # Legendre symbols, see Wikipedia
    leg = np.zeros(p,dtype=int)
    for i in range(p):
        temp = i**((p-1)/2) % p
        if temp > 1:
            temp = -1
        leg[i] = temp
    return leg[1:]                    # remove first
#--------------------------------

def Hadamard28(L,p):                       # form Hadamard 28 by 28 matrix
    
    def getValues(i,j,a,b,c,d):
        n = 2*(p+1)
        if 2*i-1 < 0 and 2*j-1 < 0:
            Had28[n-1,n-1]      = a
        else:
            Had28[i+i-1  ,j+j-1]= a
        
        if 2*i-1 < 0 and 2*j >= 0:
            Had28[n-1,j+j]      = b
        else:
            Had28[i+i-1,j+j]    = b 
            
        if 2*i >= 0 and 2*j-1 < 0:
            Had28[i+i,n-1]     = c
        else:
            Had28[i+i  ,j+j-1] = c
        
        Had28[i+i    ,j+j]   = d
    pass
        
    #----------------------------------- 
    
    Had28 = np.zeros((2*(p+1),2*(p+1)),dtype=int)
    B     = np.zeros((p+1,p+1),dtype=int)      # insert Q into B as 2nd row & second column
    Q     = np.zeros((p,p),dtype=int)
    Q[0,1:] = L[:]                         # Legendre
    #print('***',Q)
    for i in range(1,p):                   # roll vector all except first 
        Q[i,:] = np.roll(Q[0,:],i)
    Q[0,0] = 0                             # from now on is book-keeping, 
                                           # insert Q into B as 2nd row & second column  
    for i in range(p+1):
        B[0,i] = 1
        B[i,0] = 1
    for i in range(1,p+1):
        for j in range(1,p+1):
            B[i,j] = Q[i-1, j-1]
    B[0,0] = 0
    for i in range(0,p+1):                  # S matrix so fill matrix with 1& 0 instead of 1, -1
        for j in range(0,p+1):              # index [-1] is last element in array
            
            if B[i,j] == 1: 
                getValues(i,j,1,1,1,0)
                
            if B[i,j] == -1:
                getValues(i,j,0,0,0,1)

            if B[i,j] == 0: 
                getValues(i,j,1,0,0,0)
            pass
    return Had28 
#----------------------------------------
# this runs code safely  

if __name__== '__main__':

    p = 13                                        # size = 2^k(p+1), in this case k=1
    L     = legendre(p)
    Had28 = Hadamard28(L,p)

    S28 = Had28[:-1,:-1]                          # make S matrix
    
    print('print S = 27 matrix,\nsize = ',len(S28[0,:]),len(S28[:,0])  ) 
    pass

for i in range(0,2*(p+1)-1):
    print('{:4d}'.format(i),end='  |  ')
    for j in range(0,2*(p+1)-1):
        temp = S28[j,i]
        print(temp,end=' ')
    print()
print()


print S = 27 matrix,
size =  27 27
   0  |  0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 
   1  |  1 1 0 1 1 0 0 1 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 0 0 1 1 
   2  |  0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 0 1 0 1 1 0 1 0 0 1 1 0 
   3  |  1 1 1 1 0 1 1 0 0 1 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 0 0 
   4  |  0 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 0 1 0 1 1 0 1 0 0 1 
   5  |  1 0 0 1 1 1 0 1 1 0 0 1 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 
   6  |  0 0 1 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 0 1 0 1 1 0 1 0 
   7  |  1 1 1 0 0 1 1 1 0 1 1 0 0 1 1 1 1 0 0 0 0 0 0 0 0 1 1 
   8  |  0 1 0 0 1 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 0 1 0 1 1 0 
   9  |  1 1 1 1 1 0 0 1 1 1 0 1 1 0 0 1 1 1 1 0 0 0 0 0 0 0 0 
  10  |  0 1 0 1 0 0 1 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 0 1 0 1 
  11  |  1 0 0 1 1 1 1 0 0 1 1 1 0 1 1 0 0 1 1 1 1 0 0 0 0 0 0 
  12  |  0 0 1 1 0 1 0 0 1 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 0 1 
  13  |  1 0 0 0 0 1 1 1 1 0 0 1 1 1 0 1 1 0 0 1 1 1 1 0 0 0 0 
  14  |  0 0 1 0 1 1 0 1 0 0 1 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 
  15 

# End of Hadamard matrix generation¶

# Start of calculation to produce patterns for different size matrices and number of positives

In [ ]:
def make_rows(m):                         # type 2 matrix
    if m == 7:  
        letters = ['A','D','F','G']
    if m == 11:
        letters = ['A', 'C', 'G', 'H', 'I', 'K']                        #  as in table in paper
    if m == 15:
        #letters = ['A', 'B', 'C', 'D', 'E', 'F','G','H','M','N']       # best on 3 and 4 groups
        letters = ['A', 'B', 'C', 'D', 'E', 'F', 'I', 'J', 'K', 'O']    # 4 with 4 + 6 with 6.
        #letters = ['A', 'B', 'C', 'D', 'E', 'F', 'H', 'J', 'K', 'N']   # doubling method 4 x 6 + 4*4
    if m == 19:
        #letters = ['A', 'D', 'G', 'I', 'K', 'M','N','O','P','S'] 
        letters = ['A', 'C', 'D', 'I', 'K', 'M', 'N', 'O', 'P', 'S']    # all fives
    if m == 23:
        letters = ['A','F','H','I','K','O','P','R','T','V','W']
    if m == 27:
        letters = ['A','B','C','D','E','F','G','H','J','M','O','T','X']
    if m == 31:
        letters = ['C','D','G','J','L','N','P','U','V','X','Y','a','e']
    rows    = np.zeros(len(letters),dtype=int)
    for i in range(len(rows) ):
        if ord(letters[i]) < 91:                                 # ascii A = 65,  Z = 90
            rows[i] = ord(letters[i]) - 65                       # is capital A = 1, Z = 26
        else:
            rows[i] = ord(letters[i]) - 97 + 27 -1               # ascii 97 is lower case a
    return rows
#---------------------------------
def make_rows_type1(m):                # type 1 matrix 
    if m == 7:  
        letters = ['A','D','F']
    if m == 11:
        letters = ['B', 'D', 'E', 'G']                     
    if m == 15:
        letters = ['B', 'C', 'D', 'E']    
    if m == 19:   
        letters = ['B', 'E', 'F', 'H', 'N']    
    if m == 23:
        letters = ['A','H','K','N','Q']
    if m == 27:
        letters = ['A','B','E','N','V']
    if m == 31:
        letters = ['B','C','E','G','L','O']
    rows    = np.zeros(len(letters),dtype=int)
    for i in range(len(rows) ):
        if ord(letters[i]) < 91:                                 # ascii A = 65,  Z = 90
            rows[i] = ord(letters[i]) - 65                       # is capital A => 1, Z => 26
        else:
            rows[i] = ord(letters[i]) - 97 + 27 - 1               # ascii 97 is lower case a
    return rows
#---------------------------------

def make_reduced_matrix(m,rows,S):                       # Reduced S matrix, remove last column
    Sred = np.zeros((len(rows),m-1), dtype =int)
    for k,val in enumerate(rows):
        arry = S[val, 0:m - 1]
        Sred[k, 0:m - 1] = arry
        pass
    return Sred
#--------------------------

def do_row_sums(data,Sred):                   # Calculate down columns to get sum
    row, col = Sred.shape
    #print('row col', row,col)
    dotp = np.zeros(row, dtype=float)
    for i in range(row):                      # dot product. Make vector dotp
        dotp[i] = np.dot(Sred[i,:] , data[:])
    return dotp
#-----------------------------

def test_for_positives(data,Sred,pnum):
    
    dotpvec = do_row_sums(data,Sred)
    if sum(dotpvec) == 0:
        temp = np.zeros(len(dotpvec),dtype=int)# 0 positive
        indx = [-10 for i in range(10)]
        indx[0] = -2
        flag = 0
        print(temp,' Zero pos')
        pnum[0] = pnum[0] + 1
    else:                                               # samples present 
        
        pnum, q,   indx, flag,total_column,total_numbs,total_ones = one_sample(Sred,   dotpvec,comb_indx1,pnum,total_column,total_numbs,total_ones)
        if q == 0:
            pnum, q,indx,flag,total_column,total_numbs,total_ones = two_samples(Sred,  dotpvec,comb_indx2,pnum,total_column,total_numbs,total_ones)
        if q == 0:
            pnum, q,indx,flag,total_column,total_numbs,total_ones = three_samples(Sred,dotpvec,comb_indx3,pnum,total_column,total_numbs,total_ones)
        if q == 0:
            print('more than 3,  vector =',dotpvec)
            indx = [-10 for i in range(10)]
            indx[0] = -1
            pnum[7] = pnum[7] + 1
            flag = 1
    return pnum ,indx,flag                                            # number of positives in this array
#--------------------------------------
def allcombs(alist, num):
    return list(combinations(alist,num))
#---------------------------------------


# Finding patterns for different numbers of positives.

In [ ]:
#
def get_patterns(m,nc,Sred,QSred,pnum,sflag): 
    
    #-----------------------------------------
    def tostr(a):                                 # Qdots list. sort out string a for printing
        b = a.split()
        c = ''
        for i in range(len(b)):
            c = c + (  b[i].replace('0+','').replace('+','').replace(',','  ') )
        s0 = c.split()
        z = ' '
        for i in range(len(s0)):
            if len(s0[i]) > 1:
                z = z+ '   ' + s0[i].replace('0','   ')
            else: 
                z = z + '  ' + s0[i]
        return z
    #-----------------------------------------
    def one_positive(Sred,QSred,dotp_vector,w):
    
        temp = np.zeros(len(dotp_vector),dtype=int)  
        a0 = w[0]                               # a0 index for Sred array
    
        temp[:] = Sred[:,a0]                    # column a0
        total_column.append(str(temp) )
        total_numbs.append(str(a0 + 1))
        tempones = np.zeros(len(temp),dtype=int)            # if not quantitative all values >1 =1
        for k in range(len(temp)):
            tempones[k]= temp[k]
            if temp[k] >1:
                tempones[k] = 1     
        total_ones.append(str(tempones) )                    
        temp_a0 = []
        for k in range(len(temp)):  
            temp_a0.append(QSred[k,a0] )  # Q dots
        astr = ''
        for k in range(len(temp)):
            astr = astr + ','+str(temp_a0[k]) 
        total_dots.append(tostr(astr))
        
        pass
    #---------------------------------------------------
    def two_positive(Sred,QSred,dotp_vector,w):
    
        temp = np.zeros(len(dotp_vector),dtype=int)  
        a0 = w[0]                                 # a0,a1 is index for Sred array
        a1 = w[1]
        temp[:] = Sred[:,a0] + Sred[:, a1]        # two columns a0, and a1
        total_column.append(str(temp) )
        total_numbs.append(str(a0 + 1)+'  '+str(a1 + 1))
        tempones = np.zeros(len(temp),dtype=int)            # if not quantitative all values >1 =1
        for k in range(len(temp)):
            tempones[k]= temp[k]
            if temp[k] >1:
                tempones[k] = 1     
        total_ones.append(str(tempones) )                    
        temp_a0 = []
        temp_a1 = []
        for k in range(len(temp)):  
            temp_a0.append(QSred[k,a0] )  # Q dots
            temp_a1.append(QSred[k,a1] )  # Q dots 
        astr = ''
        for k in range(len(temp)):
            astr = astr + ','+str(temp_a0[k]) + '+' + str(temp_a1[k]) 
        total_dots.append(tostr(astr))
        pass
    #---------------------------------------------------
    def three_positive(Sred,QSred,dotp_vector,w):
    
        temp = np.zeros(len(dotp_vector),dtype=int)  
        a0 = w[0]                               # a0,a1 is index for Sred array
        a1 = w[1]
        a2 = w[2]
        temp[:] = Sred[:,a0] + Sred[:, a1] + Sred[:, a2]     # three columns a0,a1 and a2
        total_column.append(str(temp) )
        total_numbs.append(str(a0 + 1)+'  ' +str(a1 + 1)+'  '+ str(a2 + 1) )
        tempones = np.zeros(len(temp),dtype=int)            # if not quantitative all values >1 =1
        for k in range(len(temp)):
            tempones[k]= temp[k]
            if temp[k] >1:
                tempones[k] = 1     
        total_ones.append(str(tempones) )                    
        temp_a0 = []
        temp_a1 = []
        temp_a2 = []
        for k in range(len(temp)):  
            temp_a0.append(QSred[k,a0] )  # Q dots
            temp_a1.append(QSred[k,a1] )  # Q dots
            temp_a2.append(QSred[k,a2] )  # Q dots 
        astr = ''
        for k in range(len(temp)):
            astr = astr + ',' +str(temp_a0[k])+'+'+ str(temp_a1[k]) +'+' + str(temp_a2[k]) 
        total_dots.append(tostr(astr))
        pass
    #---------------------------------------------------
    def four_positive(Sred,QSred,dotp_vector,w):
    
        temp = np.zeros(len(dotp_vector),dtype=int )  
        a0 = w[0]                               # a0,a1 is index for Sred array
        a1 = w[1]
        a2 = w[2]
        a3 = w[3]
        temp[:] = Sred[:,a0] + Sred[:, a1] + Sred[:, a2] + Sred[:, a3]     # 4 columns a0,a1,a3 and a4
        total_column.append(str(temp) )
        total_numbs.append(str(a0 + 1)+'  ' +str(a1 + 1)+'  '+ str(a2 + 1)+'  '+ str(a3 + 1) )
        tempones = np.zeros(len(temp),dtype=int)            # if not quantitative all values >1 =1
        for k in range(len(temp)):
            tempones[k]= temp[k]
            if temp[k] >1:
                tempones[k] = 1     
        total_ones.append(str(tempones) )                    
        temp_a0 = []
        temp_a1 = []
        temp_a2 = []
        temp_a3 = []
        for k in range(len(temp)):  
            temp_a0.append(QSred[k,a0] )  # Q dots
            temp_a1.append(QSred[k,a1] )  # Q dots
            temp_a2.append(QSred[k,a2] )  # Q dots
            temp_a3.append(QSred[k,a3] )  # Q dots
        astr = ''
        for k in range(len(temp)):
            astr = astr + ',' +str(temp_a0[k])+'+'+ str(temp_a1[k]) +'+' + str(temp_a2[k])+'+' + str(temp_a3[k])  
        total_dots.append(tostr(astr))
        pass
    #---------------------------------------------------
 

    def five_positive(Sred,QSred,dotp_vector,w):
    
        temp = np.zeros(len(dotp_vector),dtype=int )  
        a0 = w[0]                               # a0,a1 is index for Sred array
        a1 = w[1]
        a2 = w[2]
        a3 = w[3]
        a4 = w[4]
        temp[:] = Sred[:,a0] + Sred[:, a1] + Sred[:, a2] + Sred[:, a3]+ Sred[:, a4]    # 5 columns a0,a1...
        total_column.append(str(temp) )
        total_numbs.append(str(a0 + 1)+'  ' +str(a1 + 1)+'  '+ str(a2 + 1)+'  '+ str(a3 + 1)+'  '+ str(a4 + 1) )
        tempones = np.zeros(len(temp),dtype=int)            # if not quantitative all values >1 =1
        for k in range(len(temp)):
            tempones[k]= temp[k]
            if temp[k] >1:
                tempones[k] = 1     
        total_ones.append(str(tempones) )                    
        temp_a0 = []
        temp_a1 = []
        temp_a2 = []
        temp_a3 = []
        temp_a4 = []
        for k in range(len(temp)):  
            temp_a0.append(QSred[k,a0] )  # Q dots
            temp_a1.append(QSred[k,a1] )  # Q dots
            temp_a2.append(QSred[k,a2] )  # Q dots
            temp_a3.append(QSred[k,a3] )  # Q dots
            temp_a4.append(QSred[k,a4] )  # Q dots
        astr = ''
        for k in range(len(temp)):
            astr = astr + ',' +str(temp_a0[k])+'+'+ str(temp_a1[k]) +'+' + str(temp_a2[k])+'+' + str(temp_a3[k]) +'+' + str(temp_a4[k]) 
        total_dots.append(tostr(astr))
        pass
    #---------------------------------------------------

    def pseudo_inv(Sred,dotp_vec):        # use library pseudo inversion of matrix. ( Moore-Penrose algorithm)
        psi = La.pinv(Sred) @ dotp_vec    # do inverse
        mx = max(abs(psi))                
        for i in range(len(psi)):
            if psi[i] < 0:
                psi[i] = 0
        temp = np.array([round(100*x/mx) for x in psi])  # normalise, make -ve zero
        return temp
    #-----------------------------------
# top level of code in this def
    total_column  = []
    total_numbs   = []
    total_ones    = []
    total_pseudo  = []
    total_dots    = []
    
    if nc == 1:
        comb_indx1 = allcombs([i for i in range(m-1)], nc)
        lenc = len(comb_indx1)                                 
    if nc == 2:
        comb_indx2 = allcombs([i for i in range(m-1)], nc)
        lenc = len(comb_indx2)
    if nc == 3:
        comb_indx3 = allcombs([i for i in range(m-1)], nc)
        lenc = len(comb_indx3)
    if nc == 4:
        comb_indx4 = allcombs([i for i in range(m-1)], nc)
        lenc = len(comb_indx4)
    if nc == 5:
        comb_indx5 = allcombs([i for i in range(m-1)], nc)
        lenc = len(comb_indx5)
    
    #print('Initial S matrix has', m, ' columns. Number of combinations = ',lenc,'\n' )
    for i in range(lenc):                        # Data set to test with
        data = np.zeros(m-1,dtype=int) 
        
        if nc == 1:
            w = comb_indx1[i]                    # combinations  
            data[w[0]] = 1
            dotp_vector  = do_row_sums(data,Sred) 
            one_positive(Sred,QSred,dotp_vector,w)
            atemp = pseudo_inv(Sred,dotp_vector)
            total_pseudo.append(np.array2string(atemp, precision=3,suppress_small=True ))
        
    
        if nc == 2:
            w = comb_indx2[i]
            data[w[0]] = 1
            data[w[1]] = 1
            dotp_vector  = do_row_sums(data,Sred)
            two_positive(Sred,QSred,dotp_vector,w)
            atemp = pseudo_inv(Sred,dotp_vector)
            total_pseudo.append(np.array2string(atemp, precision=3,suppress_small=True ))
            
        if nc == 3:
            w = comb_indx3[i]
            data[comb_indx3[i][0]] = 1
            data[comb_indx3[i][1]] = 1
            data[comb_indx3[i][2]] = 1
            dotp_vector  = do_row_sums(data,Sred)
            three_positive(Sred,QSred,dotp_vector,w)
            atemp = pseudo_inv(Sred,dotp_vector)
            total_pseudo.append(np.array2string(atemp, precision=3,suppress_small=True ))

        if nc == 4:
            w = comb_indx4[i]
            data[comb_indx4[i][0]] = 1
            data[comb_indx4[i][1]] = 1
            data[comb_indx4[i][2]] = 1
            data[comb_indx4[i][3]] = 1 
            dotp_vector  = do_row_sums(data,Sred)
            four_positive(Sred,QSred,dotp_vector,w)
            atemp = pseudo_inv(Sred,dotp_vector)
            total_pseudo.append(np.array2string(atemp, precision=3,suppress_small=True ))
            
        if nc == 5:
            w = comb_indx5[i]
            data[comb_indx5[i][0]] = 1
            data[comb_indx5[i][1]] = 1
            data[comb_indx5[i][2]] = 1
            data[comb_indx5[i][3]] = 1 
            data[comb_indx5[i][4]] = 1
            dotp_vector  = do_row_sums(data,Sred)
            five_positive(Sred,QSred,dotp_vector,w)
            atemp = pseudo_inv(Sred,dotp_vector)
            total_pseudo.append(np.array2string(atemp, precision=3,suppress_small=True ))
            
        pass

    return total_column,total_numbs,total_ones, total_pseudo,total_dots,lenc

#-----------------------------------------------------------------

def make_QSred(Sred):
    r,c = Sred.shape
    QSred = np.zeros((r,c),dtype=int)
    QSred[:,:] = Sred[:,:]
    pattern = np.array([1,2,3])
    atile = np.tile(pattern,2*c)
    for i in range(c):
        QSred[:,i] = Sred[:,i]*atile[i]
    return QSred

#-----------------------------------------
def sort_string(zs):
    for i in range(len(zs)):
        ls = len(zs[i])
        if ls > 1:
            s = sorted([ zs[i][k] for k in range(ls)] )
            zs[i] = ''.join(s)
    return zs
        #-------------------------------
def print_all(total_column,total_numbs,total_dots):        # Quantitative 0,1,2..
    
    print('Sorted list. Type 2 matrices with quantitative testing.')
    print('** indicates duplicate, 0 absence of signal, in Q-dots, 123,234 etc are  \u03BB1 & \u03BB2 & \u03BB3')
    print('\n   Column pattern : posn of +ve : colm sum:      Qdots \u03BB index\n' )
    
    indx = np.argsort(total_column )
    star = '  '
    atemp = total_column[indx[0]]
    zz =''.join(i for i in atemp if i in '0123456789') 
    print('{:4d}{:s}{:10s}{:2s}{:<15s}{:>4d}{:s} '
            .format(1,':',atemp, star, total_numbs[indx[0]],sum([int(ord(i)-48) for i in zz]),':' ),end='' ) 
    zs = total_dots[indx[0]].split()
    zs = sort_string(zs)
    for k in range(len(zs)):
        print('{:6s}'.format(zs[k]),end='')
    print()
    for i in range(1,len(total_column),1 ):
        asc = indx[i]                   # index after sorting
        zz =''.join(i for i in total_column[asc] if i in '0123456789')
        star ='  '
        if atemp == total_column[asc]:
            star = '**'
        atemp = total_column[asc]
        s = sum([int(ord(k)-48) for k in zz] ) 
        print('{:4d}{:s}{:10s}{:2s}{:<15s}{:>4d}{:s} ' 
              .format(i+1,':',total_column[asc], star, total_numbs[asc],s,':') ,end='')
        zs = total_dots[asc].split()
        zs = sort_string(zs)
    
        for k in range(len(zs)):
            print('{:6s}'.format(zs[k]),end='')
        print()
    
#----------------------------------------- 
def print_Qdots(total_column,total_numbs,total_dots):        # Quantitative 0,1,2..
    
    print('\nSorted list based on emission index, 12 13 etc. Type 2 matrices with emission testing.' )
    print('** indicates duplicate, 0 absence of signal, in Q-dots, 123,234 etc are  \u03BB1 & \u03BB2 & \u03BB3')
    print('\n        column pattern     posn of +ve :     Q-dots \u03BB index\n' )
    

    indx = np.argsort(total_dots )  # sorted on dots wavelength index, 1,2,3 
    atemp = total_dots[indx[0]]
    
    star = '  '
    print('{:4d}{:s} {:2s} {:s} {:<15s} {:s}  ' 
            .format(1,':', star, total_column[indx[0]],total_numbs[indx[0]],':' ),end='' ) 
    zs = total_dots[indx[0]].split()
    zs = sort_string(zs)
    for k in range(len(zs)):
        print('{:6s}'.format(zs[k]),end='')
    print()
    for i in range(1,len(total_dots),1 ):
        asc = indx[i]                                  # index after sorting
        star ='  '
        if atemp == total_dots[asc]:
            star = '**'
        atemp = total_dots[asc]
        print('{:4d}{:s} {:2s} {:s} {:<15s} {:s}  ' 
              .format(i+1,':', star, total_column[asc],total_numbs[asc],':' ),end='' )
        zs = total_dots[asc].split()
        zs = sort_string(zs)
        
        for k in range(len(zs)):
                print('{:6s}'.format(zs[k]),end='')
        print()
    
#----------------------------------------- 

def print_typeN(total_ones,total_numbs):  # not quantitative so 1 and 0 only
    print('\nSorted. Type 2 matrix. Not quantitative testing so many duplicates expected\n')

    indx = np.argsort(total_ones )
    ntemp = []
    for i in range(len(total_numbs)):
        ntemp.append(total_ones[indx[i]] )
    stemp = sorted(ntemp)
    i = 0
    while i < len(stemp):
        print('{:>4d}{:s}  {:>8s} '.format(i+1,':', stemp[i]), end='   ')    
        j = 0
        for k in range(len(ntemp)):
            if stemp[i] == ntemp[k]:
                print(' {:8s}'.format(total_numbs[indx[k]]),end='   ')
                j = j + 1
        i = i + j
        print()
#-----------------------------------

def print_pseudo(total_pseudo,total_column,total_numbs):
    
    indx = np.argsort(total_pseudo)
    np.set_printoptions(linewidth=200)
    sort_pseudo = sorted(total_pseudo)
    print('\nPseudo Inversion. Quantitative type 2 matrix.\nMaximum normalised to 100, values rounded and negatives made zero\n')
    print('Columns are;  Vectors (sorted & normalised), pattern of results and positions of positives\n')
    stemp = sort_pseudo[0]
    print('{:4d}{:s}{:50s} {:22s} {:15s}'.format(0,':', stemp , total_column[indx[0]],total_numbs[indx[0]]) )
    for i in range(1,len(total_pseudo),1 ):
        astr = sort_pseudo[i]
        print('{:4d}{:s}{:50s} {:22s} {:15s}'.format( i+1,':', astr , total_column[indx[i]], total_numbs[indx[i]]) )
        if stemp == astr: 
            print('*** recheck')
        stemp = astr
    
#------------------------------------
#import os ; os.system('say "Beer time."')
#os.system('say "finished now. "')

# Widgets to list reduced matrices, patterns and positives

In [ ]:
# try and put the above calc into widgets
#-------------------------------------

def Go_button_clicked(b):                      # read values and do calculation

    prnt = int(radio_print.value)
    nc   = int(radio_numpos.value)
    m    = int(radio_matrix.value)
    with outpt:
        print('matrix size = ', m,'number of positives = ', nc)
    
    rows = make_rows(m)                         # make_rows()  is defined above
    
    if m in [7, 11, 19, 23, 31]:                # make reduced S matrix
        S = quadratic_hadamard(m)
    
    if m == 15:
        S = shift_hadamard(4)                   # k= 4 is fixed as 2^k-1 for m = 15 
    
    if m == 27:
        p = 13                                  # size = 2^k(p+1), in this case k=1
        L = legendre(p)
        H28 = Hadamard28(L,p)
        S = Had28[:-1,:-1]
    Sred = make_reduced_matrix(m,rows,S)        # make_reduced_matrix is defined above
    QSred= make_QSred(Sred)                     # reduced matric with 123123 pattern in columns
    with outpt:
        print('S matrix\n', S,'\n')
        print('Reduced S matrix\n',Sred,'\n')
        print('Reduced patterened S matrix\n',QSred,'\n')
        print('Calculating ...')
      
    pnum = np.zeros(10,dtype=int)               # array for number of positives
    sflag = 0
    total_column,total_numbs,total_ones, total_pseudo,total_dots,lenc = get_patterns(m,nc,Sred, QSred,pnum, sflag )
    
    with outpt:
        print('Initial S matrix has', m, ' columns. Number of combinations = ',lenc,'\n' )
        if prnt == 1:
            print_all(total_column,total_numbs,total_dots)
            print_typeN(total_ones,total_numbs)
            print_pseudo(total_pseudo,total_column,total_numbs)
            print_Qdots(total_column,total_numbs,total_dots)
        if prnt == 2:
            print_all(total_column,total_numbs,total_dots)
        if prnt == 3:
            print_typeN(total_ones,total_numbs)
        if prnt == 4:
            print_pseudo(total_pseudo,total_column,total_numbs)
        if prnt == 5:
            print_Qdots(total_column,total_numbs,total_dots)
#---------------------------------------
def clr_button_clicked(b):
    outpt.clear_output()
            
#---------------------------------------
def make_boxes():
    vbox2 = wgt.VBox([wgt.Label('Choose what to print'), radio_print, bGo, bclr])
    vbox1 = wgt.VBox([wgt.Label('Choose matrix size and number of positives'), L0,radio_numpos, radio_matrix])
    return  vbox1,vbox2
#----------------------------------
L0 = wgt.Label('Larger matrices will produce hundreds of lines of output',layout=Layout(width='400px',height='20px') )

radio_print   = wgt.RadioButtons(value= 1   ,
        options=[('All results',1), ('Quantitative, list 012201..',2), ('Not quantitative list 011101..',3),
                 ('Pseudo invert + lists',4),('Quantitative. \u03BB index',5)], description='type 2 matrix')
radio_numpos  = wgt.RadioButtons(value= '2' , options=['1', '2', '3','4'], description=' positives')
radio_matrix  = wgt.RadioButtons(value= '11', options=['7', '11', '15', '19', '23','27'], description='S matrix size')

bGo    = wgt.Button(description='GO',    style = dict(button_color='lightgreen',font_weight='bold'))
bGo.on_click(Go_button_clicked)

bclr    = wgt.Button(description='Clear',style = dict(button_color='lightblue',font_weight='bold'))
bclr.on_click(clr_button_clicked)

outpt = wgt.Output(layout={'border': '1px solid red'})

vbox1,vbox2 = make_boxes()
display(wgt.HBox([ vbox1,vbox2]), outpt)


Output(layout=Layout(border='1px solid red'))